# Introduction to Data Science 2025

# Week 4

In this week's exercise, we look at prompting and zero- and few-shot task settings. Below is a text generation example from https://github.com/TurkuNLP/intro-to-nlp/blob/master/text_generation_pipeline_example.ipynb demonstrating how to load a text generation pipeline with a pre-trained model and generate text with a given prompt. Your task is to load a similar pre-trained generative model and assess whether the model succeeds at a set of tasks in zero-shot, one-shot, and two-shot settings.

**Note: Downloading and running the pre-trained model locally may take some time. Alternatively, you can open and run this notebook on [Google Colab](https://colab.research.google.com/), as assumed in the following example.**

## Text generation example

This is a brief example of how to run text generation with a causal language model and `pipeline`.

Install [transformers](https://huggingface.co/docs/transformers/index) python package. This will be used to load the model and tokenizer and to run generation.

In [1]:
!pip install --quiet transformers

Import the `AutoTokenizer`, `AutoModelForCausalLM`, and `pipeline` classes. The first two support loading tokenizers and generative models from the [Hugging Face repository](https://huggingface.co/models), and the last wraps a tokenizer and a model for convenience.

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

Load a generative model and its tokenizer. You can substitute any other generative model name here (e.g. [other TurkuNLP GPT-3 models](https://huggingface.co/models?sort=downloads&search=turkunlp%2Fgpt3)), but note that Colab may have issues running larger models.

In [3]:
MODEL_NAME = 'TurkuNLP/gpt3-finnish-large'

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/218 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/562 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.53G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.53G [00:00<?, ?B/s]

Instantiate a text generation pipeline using the tokenizer and model.

In [4]:
pipe = pipeline(
    'text-generation',
    model=model,
    tokenizer=tokenizer,
    device=model.device
)

Device set to use cpu


We can now call the pipeline with a text prompt; it will take care of tokenizing, encoding, generation, and decoding:

In [5]:
output = pipe('Terve, miten menee?', max_new_tokens=25)

print(output)

[{'generated_text': 'Terve, miten menee?”\n”Kiitos, ihan hyvin.”\n”Oletko kunnossa?”\n”Olen.”\n”Näytät siltä kuin olisit pyör'}]


Just print the text

In [6]:
print(output[0]['generated_text'])

Terve, miten menee?”
”Kiitos, ihan hyvin.”
”Oletko kunnossa?”
”Olen.”
”Näytät siltä kuin olisit pyör


We can also call the pipeline with any arguments that the model `generate` function supports. For details on text generation using `transformers`, see e.g. [this tutorial](https://huggingface.co/blog/how-to-generate).

Example with sampling and a high `temperature` parameter to generate more chaotic output:

In [7]:
output = pipe(
    'Terve, miten menee?',
    do_sample=True,
    temperature=10.0,
    max_new_tokens=25
)

print(output[0]['generated_text'])

Terve, miten menee? – Sanna Ukkomaa.
“On hyvin tyypillistä kuulla: Aiot erota miehestä! Ihan hirveää…
Mitä voin auttaa – Timo Heinonen


## Exercise 1

Your task is to assess whether a generative model succeeds in the following tasks in zero-shot, one-shot, and two-shot settings:

- binary sentiment classification (positive / negative)

- person name recognition

- two-digit addition (e.g. 11 + 22 = 33)

For example, for assessing whether a generative model can name capital cities, we could use the following prompts:

- zero-shot:
	>"""\
	>Identify the capital cities of countries.
	>
	>Question: What is the capital of Finland?\
	>Answer:\
	>"""
- one-shot:
	>"""\
	>Identify the capital cities of countries.
	>
	>Question: What is the capital of Sweden?\
	>Answer: Stockholm
	>
	>Question: What is the capital of Finland?\
	>Answer:\
	>"""
- two-shot:
	>"""\
	>Identify the capital cities of countries.
	>
	>Question: What is the capital of Sweden?\
	>Answer: Stockholm
	>
	>Question: What is the capital of Denmark?\
	>Answer: Copenhagen
	>
	>Question: What is the capital of Finland?\
	>Answer:\
	>"""

You can do the tasks either in English or Finnish and use a generative model of your choice from the Hugging Face models repository, for example the following models:

- English: `gpt2-large`
- Finnish: `TurkuNLP/gpt3-finnish-large`

You can either come up with your own instructions for the tasks or use the following:

- English:
	- binary sentiment classification: "Do the following texts express a positive or negative sentiment?"
	- person name recognition: "List the person names occurring in the following texts."
	- two-digit addition: "This is a first grade math exam."
- Finnish:
	- binary sentiment classification: "Ilmaisevatko seuraavat tekstit positiivista vai negatiivista tunnetta?"
	- person name recognition: "Listaa seuraavissa teksteissä mainitut henkilönnimet."
	- two-digit addition: "Tämä on ensimmäisen luokan matematiikan koe."

Come up with at least two test cases for each of the three tasks, and come up with your own one- and two-shot examples.

In [13]:
# Use this cell for your code
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Choose model
model_name = "gpt2-large"  # Or "TurkuNLP/gpt3-finnish-large" for Finnish
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

def generate_answer(prompt, max_length=50):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    output = model.generate(**inputs, max_length=inputs["input_ids"].shape[1] + max_length)
    text = tokenizer.decode(output[0], skip_special_tokens=True)
    return text[len(prompt):].strip()

# -----------------------------
# Task 1: Sentiment Classification
# -----------------------------
zero_shot_prompt = """Do the following texts express a positive or negative sentiment?

Text: I love this movie! It's fantastic.
Answer:"""

one_shot_prompt = """Do the following texts express a positive or negative sentiment?

Text: I hate waiting in long lines.
Answer: negative

Text: The food was cold and tasteless.
Answer:"""

two_shot_prompt = """Do the following texts express a positive or negative sentiment?

Text: I love sunny days.
Answer: positive

Text: The service at the restaurant was terrible.
Answer: negative

Text: I enjoy reading books in my free time.
Answer:"""

# Generate answers
print("Zero-shot sentiment:", generate_answer(zero_shot_prompt))
print("One-shot sentiment:", generate_answer(one_shot_prompt))
print("Two-shot sentiment:", generate_answer(two_shot_prompt))

# -----------------------------
# Task 2: Person Name Recognition
# -----------------------------
zero_shot_names = """List the person names occurring in the following text.

Text: Alice and Bob went to the park.
Answer:"""

one_shot_names = """List the person names occurring in the following text.

Text: Sarah and Michael are playing soccer.
Answer: Sarah, Michael

Text: Yesterday, John met Maria at the library.
Answer:"""

two_shot_names = """List the person names occurring in the following text.

Text: Tom and Jerry were chasing each other.
Answer: Tom, Jerry

Text: Emily and Robert went to the museum.
Answer: Emily, Robert

Text: Alice and Bob went to the park.
Answer:"""

print("Zero-shot names:", generate_answer(zero_shot_names))
print("One-shot names:", generate_answer(one_shot_names))
print("Two-shot names:", generate_answer(two_shot_names))

# -----------------------------
# Task 3: Two-digit Addition
# -----------------------------
zero_shot_math = """This is a first grade math exam.

Question: 23 + 47 =
Answer:"""

one_shot_math = """This is a first grade math exam.

Question: 12 + 34 =
Answer: 46

Question: 58 + 36 =
Answer:"""

two_shot_math = """This is a first grade math exam.

Question: 11 + 22 =
Answer: 33

Question: 45 + 13 =
Answer: 58

Question: 23 + 47 =
Answer:"""

print("Zero-shot math:", generate_answer(zero_shot_math))
print("One-shot math:", generate_answer(one_shot_math))
print("Two-shot math:", generate_answer(two_shot_math))


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Zero-shot sentiment: I love this movie! It's fantastic.

Text: I love this movie! It's fantastic.

Answer: I love this movie! It's fantastic.

Text: I love this movie! It's fantastic.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


One-shot sentiment: positive

Text: I hate waiting in long lines.

Answer: negative

Text: I hate waiting in long lines.

Answer: negative

Text: I hate waiting in long lines.

Answer: negative


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Two-shot names: Alice, Bob

Text: Tom and Jerry went to the park.

Answer: Tom, Jerry

Text: Emily and Robert went to the museum.

Answer: Emily, Robert

Text: Alice and Bob went to


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Zero-shot math: 23 + 47 =

Question: 23 + 47 =

Answer: 23 + 47 =

Question: 23 + 47 =

Answer: 23 + 47 =

Question: 23 + 47 =

Answer: 23 +


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


One-shot math: 48

Question: 60 + 36 =

Answer: 48

Question: 60 + 36 =

Answer: 48

Question: 60 + 36 =

Answer: 48

Question: 60 + 36 =
Two-shot math: 63

Question: 11 + 22 =

Answer: 33

Question: 45 + 13 =

Answer: 58

Question: 23 + 47 =

Answer: 63

Question: 11 + 22 =


**Submit this exercise by submitting your code and your answers to the above questions as comments on the MOOC platform. You can return this Jupyter notebook (.ipynb) or .py, .R, etc depending on your programming preferences.**